### Movies for vorticity - Aids to visualize the flow

In [1]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

import matplotlib as mpl
#%matplotlib inline
from math import *

import scipy.io

import scipy as spy

from netCDF4 import Dataset

import pylab as pl

import os 

import sys

import seaborn as sns

In [2]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 


In [90]:
CanyonGrid='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run02/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

CanyonState='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run02/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)



In [91]:
#for dimobj in CanyonStateOut.variables.values():
#    print dimobj

In [92]:
# General input

nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 

z = CanyonStateOut.variables['Z']
Time = CanyonStateOut.variables['T']

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')

hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')

hFacW = rout.getField(CanyonGrid, 'HFacW') # (Z,Y,Xp1) for zonal fluxes
MaskW = rout.getMask(CanyonGrid,'HFacW') 

hFacS = rout.getField(CanyonGrid, 'HFacS') # (Z,Yp1,X) for meridional fluxes
MaskS = rout.getMask(CanyonGrid,'HFacS') 


In [93]:
# Load velocity components
uu = rout.getField(CanyonState,'U')
vv = rout.getField(CanyonState,'V')

U,V = rout.unstagger(uu,vv)

In [94]:
print(np.max(uu), np.min(uu))
print(np.max(vv), np.min(vv))

Umax = np.max(uu)

(0.45190895, -0.15401502)
(0.1515148, -0.16446754)


In [95]:
import matplotlib.animation as animation

In [96]:
sns.set()
sns.set_style('white')
sns.set_context("talk")
#spdcmap = sns.cubehelix_palette(light=1, as_cmap=True)
spdcmap = 'RdYlGn_r'

In [97]:
def rel_vort(x,y,u,v):
    """-----------------------------------------------------------------------------
    rel_vort calculates the z component of relative vorticity.
    
    INPUT:
    x,y,u,v should be at least 2D arrays in coordinate order (..., Y , X ) 
    
    OUTPUT:
    relvort - z-relative vorticity array of size u[...,2:-2,2:-2]
    -----------------------------------------------------------------------------"""
    
    dvdx = (v[...,1:-1, 2:]-v[...,1:-1, :-2])/(x[...,1:-1, 2:]-x[...,1:-1, :-2])
    dudy = (u[...,2:, 1:-1]-u[..., :-2,1:-1])/(y[..., 2:,1:-1]-y[..., :-2,1:-1])
    relvort = dvdx - dudy
    return relvort

In [98]:
def Plot1(t,ax1):    
    uplane = np.ma.array(U[t,zslice1,:,1:],mask=MaskC[zslice1,1:,1:])
    vplane = np.ma.array(V[t,zslice1,1:,:],mask=MaskC[zslice1,1:,1:])
    vort = rel_vort(xc[1:,1:],yc[1:,1:],uplane,vplane)
    csSpd = np.linspace(-0.00034,0.00034,num=25)
    ax1.clear()
    mesh=ax1.contourf(xc[145+1:265+1,2:-1],yc[145+1:265+1,2:-1],vort[145:265,:],csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax1)
    CS = ax1.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[147],colors='k',linewidths=[2.0] )
    vect = ax1.quiver(xc[145:265:5, 1::20], yc[145:265:5, 1::20], uplane[145:265:5, ::20]/Umax, vplane[145:265:5, ::20]/Umax
                      , color='k')
    
    ax1.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax1.set_xlabel('Alongshore distance [km]')
    ax1.set_xticks([20000,40000,60000,80000,100000])
    ax1.set_xticklabels(['20', '40', '60', '80', '100'])
    ax1.set_ylabel('Across-shore distance [km]')
    ax1.set_yticks([40000,45000,50000,55000])
    ax1.set_yticklabels(['40', '45', '50', '55'])
    ax1.set_title('Relative vorticity at %0.2f m , day %0.1f' %(z[zslice1], (t/2.0)))

def Plot2(t,ax2):    
    uplane = np.ma.array(U[t,zslice2,:,1:],mask=MaskC[zslice2,1:,1:])
    vplane = np.ma.array(V[t,zslice2,1:,:],mask=MaskC[zslice2,1:,1:])
    vort = rel_vort(xc[1:,1:],yc[1:,1:],uplane,vplane)
    csSpd = np.linspace(-0.00034,0.00034,num=25)
    ax2.clear()
    mesh=ax2.contourf(xc[145+1:265+1,2:-1],yc[145+1:265+1,2:-1],vort[145:265,:],csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax2)
    CS = ax2.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[147],colors='k',linewidths=[2.0] )
    vect = ax2.quiver(xc[145:265:5, 1::20], yc[145:265:5, 1::20], uplane[145:265:5, ::20]/Umax, vplane[145:265:5, ::20]/Umax
                      , color='k')
    
    ax2.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax2.set_xlabel('Alongshore distance [km]')
    ax2.set_xticks([20000,40000,60000,80000,100000])
    ax2.set_xticklabels(['20', '40', '60', '80', '100'])
    ax2.set_ylabel('Across-shore distance [km]')
    ax2.set_yticks([40000,45000,50000,55000])
    ax2.set_yticklabels(['40', '45', '50', '55'])
    ax2.set_title('Relative vorticity at %0.2f m' %z[zslice2])

def Plot3(t,ax3):    
    uplane = np.ma.array(U[t,zslice3,:,1:],mask=MaskC[zslice3,1:,1:])
    vplane = np.ma.array(V[t,zslice3,1:,:],mask=MaskC[zslice3,1:,1:])
    vort = rel_vort(xc[1:,1:],yc[1:,1:],uplane,vplane)
    csSpd = np.linspace(-0.00035,0.00035,num=25)
    ax3.clear()
    mesh=ax3.contourf(xc[145+1:265+1,2:-1],yc[145+1:265+1,2:-1],vort[145:265,:],csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax3)
    CS = ax3.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[147],colors='k',linewidths=[2.0] )
    vect = ax3.quiver(xc[145:265:5, 1::20], yc[145:265:5, 1::20], uplane[145:265:5, ::20]/Umax, vplane[145:265:5, ::20]/Umax
                      , color='k')
    
    ax3.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax3.set_xlabel('Alongshore distance [km]')
    ax3.set_xticks([20000,40000,60000,80000,100000])
    ax3.set_xticklabels(['20', '40', '60', '80', '100'])
    ax3.set_ylabel('Across-shore distance [km]')
    ax3.set_yticks([40000,45000,50000,55000])
    ax3.set_yticklabels(['40', '45', '50', '55'])
    ax3.set_title('Relative vorticity at %0.2f m' %z[zslice3])

def Plot4(t,ax4):    
    uplane = np.ma.array(U[t,zslice4,:,1:],mask=MaskC[zslice4,1:,1:])
    vplane = np.ma.array(V[t,zslice4,1:,:],mask=MaskC[zslice4,1:,1:])
    vort = rel_vort(xc[1:,1:],yc[1:,1:],uplane,vplane)
    csSpd = np.linspace(-0.00034,0.00034,num=25)
    ax4.clear()
    mesh=ax4.contourf(xc[145+1:265+1,2:-1],yc[145+1:265+1,2:-1],vort[145:265,:],csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax4)
    CS = ax4.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[147],colors='k',linewidths=[2.0] )
    vect = ax4.quiver(xc[145:265:5, 1::20], yc[145:265:5, 1::20], uplane[145:265:5, ::20]/Umax, vplane[145:265:5, ::20]/Umax
                      , color='k')
    
    ax4.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax4.set_xlabel('Alongshore distance [km]')
    ax4.set_xticks([20000,40000,60000,80000,100000])
    ax4.set_xticklabels(['20', '40', '60', '80', '100'])
    ax4.set_ylabel('Across-shore distance [km]')
    ax4.set_yticks([40000,45000,50000,55000])
    ax4.set_yticklabels(['40', '45', '50', '55'])
    ax4.set_title('Relative vorticity at %0.2f m' %z[zslice4])


In [99]:
## Animation
#N=5
xslice=180
yslice=235
zslice1= 15 
zslice2= 24 
zslice3= 29 # shelf break index
zslice4= 33 

#Empty figures
fig,((ax1,ax2),(ax3, ax4))= plt.subplots(2, 2, figsize=(18, 12))

#Initial image
def init():
    Plot1(0,ax1)
    Plot2(0,ax2)
    Plot3(0,ax3)
    Plot4(0,ax4)
    
    
    plt.tight_layout()
    #return[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]
    

def animate(tt):
    Plot1(tt,ax1)
    Plot2(tt,ax2)
    Plot3(tt,ax3)
    Plot4(tt,ax4)
    
    
    plt.tight_layout()
#The animation function (max frames=47)   


anim = animation.FuncAnimation(fig, animate, init_func=init,frames=18, interval = 200,blit=False, repeat=False)

##A line that makes it all work
mywriter = animation.FFMpegWriter()

##Save in current folder
anim.save('Vort_3DDIFF_run02.mp4',writer=mywriter,fps=0.2, dpi = 300, bitrate = 1000000, codec = "libx264")

plt.show()



[[-4.81306756920906e-06 -4.812143357447648e-06 -4.811075740523128e-06 ...,
  -4.816991287254473e-06 -4.81601223831029e-06 -4.815143862845296e-06]
 [-5.408844333573336e-06 -5.408979175087809e-06 -5.409014033619401e-06 ...,
  -5.411781484724782e-06 -5.41075867661367e-06 -5.409888840691513e-06]
 [-5.871989838591586e-06 -5.872339938753922e-06 -5.872719266467937e-06 ...,
  -5.8744208981143455e-06 -5.873524413280235e-06 -5.8727596730414195e-06]
 ..., 
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]]
